In [ ]:
# To create the dataset and to import the images to the dataset

In [ ]:
#pip install google-cloud-automl

In [ ]:
#!pip install google-cloud-automl==2.5.2

In [7]:
#pip freeze > requirements.txt

In [1]:
from google.cloud import automl
#from google.cloud import automl_v1beta1 as automl

In [2]:
project_id = "gebu-data-ml-day0-01-333910"
display_name = "usecase3"
client = automl.AutoMlClient()

In [3]:
# To create the empty dataset for holding the training data

# To identify the GCP location 
project_location = f"projects/gebu-data-ml-day0-01-333910/locations/us-central1"

metadata = automl.ImageClassificationDatasetMetadata(
    classification_type=automl.ClassificationType.MULTICLASS)

dataset = automl.Dataset(display_name=display_name,
    image_classification_dataset_metadata=metadata,)

# To create dataset with the dataset in the region
response = client.create_dataset(parent = project_location, dataset = dataset, timeout = 300)

created_dataset = response.result()

print("Dataset name: {}".format(created_dataset.name))
print("Dataset id: {}".format(created_dataset.name.split("/")[-1]))

Dataset_id1= created_dataset.name.split("/")[-1]



Dataset name: projects/289649376723/locations/us-central1/datasets/ICN9208678713216991232
Dataset id: ICN9208678713216991232


In [4]:
print(Dataset_id1)

ICN9208678713216991232


In [5]:
# To load the data into the newly created dataset

project_id = "gebu-data-ml-day0-01-333910"

dataset_id = Dataset_id1
path = "gs://usecase3-vcm/AutoML_Training/data.csv"
# To access the complete path of the dataset

dataset_full_id = client.dataset_path(project_id, "us-central1", dataset_id)
    
# Access multiple inputs from the Google Cloud Storage
input_data = path.split(",")
gcs_source = automl.GcsSource(input_uris=input_data)
input_config = automl.InputConfig(gcs_source=gcs_source)
    
# To import the data from the GCS storage
response = client.import_data(name=dataset_full_id, input_config=input_config)

print("Processing import...")
#print("Data imported. {}".format(response.result()))
#print(response.result())
print(response.result())
    

Processing import...



In [6]:
# To train the model 
metadata = automl.ImageClassificationModelMetadata(
    train_budget_milli_node_hours=24000)

model = automl.Model(display_name = display_name, 
                    dataset_id = dataset_id,
                    image_classification_model_metadata = metadata,)

# To create a model with model metadata in the region
response = client.create_model(parent = project_location, model = model)

print("Training operation name: {}".format(response.operation.name))
print("Training started...")

Training operation name: projects/289649376723/locations/us-central1/operations/ICN5862735912156266496
Training started...


In [7]:
print(type(response))

<class 'google.api_core.operation.Operation'>


In [15]:
request = automl.ListModelsRequest(parent=project_location, filter="")
response1 = client.list_models(request=request)

In [16]:
# To list the model 
for model in response1:
    # Display the model information.
    if model.deployment_state == automl.Model.DeploymentState.DEPLOYED:
        deployment_state = "deployed"
    else:
        deployment_state = "undeployed"

    print("Model name: {}".format(model.name))
    print("Model id: {}".format(model.name.split("/")[-1]))
    print("Model display name: {}".format(model.display_name))
    print("Model create time: {}".format(model.create_time))
    print("Model deployment state: {}".format(deployment_state))

Model name: projects/289649376723/locations/us-central1/models/ICN7031365864511242240
Model id: ICN7031365864511242240
Model display name: usecase3
Model create time: 2021-12-28 07:02:54.384263+00:00
Model deployment state: undeployed
Model name: projects/289649376723/locations/us-central1/models/ICN211016072579514368
Model id: ICN211016072579514368
Model display name: usecase3
Model create time: 2021-12-21 00:27:26.837452+00:00
Model deployment state: deployed
Model name: projects/289649376723/locations/us-central1/models/ICN6699225391992668160
Model id: ICN6699225391992668160
Model display name: usecase3
Model create time: 2021-12-27 13:33:51.419619+00:00
Model deployment state: undeployed
Model name: projects/289649376723/locations/us-central1/models/ICN3126393143860133888
Model id: ICN3126393143860133888
Model display name: Mnist_20211213035622
Model create time: 2021-12-13 10:28:18.570378+00:00
Model deployment state: deployed


In [17]:
response1.model[0].name.split("/")[-1]

'ICN7031365864511242240'

In [19]:
# To Evaluate the model
#client = automl.AutoMLClient()
#model_id = response1.model[0].name.split("/")[-1]
model_id = "ICN6699225391992668160"
model_full_id = client.model_path(project_id, "us-central1", model_id)
print("List of model evaluations:")
#for evaluation in client.list_model_evaluations(parent=model_full_id, filter=""):
##    print("Model evaluation name: {}".format(evaluation.name))
#    print("Model annotation spec id: {}".format(evaluation.annotation_spec_id))
#    print("Create Time: {}".format(evaluation.create_time))
#    print("Evaluation example count: {}".format(evaluation.evaluated_example_count))
#    print("Classification model evaluation metrics: {}".format(evaluation.classification_evaluation_metrics))

List of model evaluations:


In [20]:
print(model_full_id)

projects/gebu-data-ml-day0-01-333910/locations/us-central1/models/ICN6699225391992668160


In [21]:
# Deployment
request2 = automl.DeployModelRequest(name = model_id)
response2 = client.deploy_model(name = model_full_id)

In [22]:
print(request2)

name: "ICN6699225391992668160"



In [ ]:
#Prediction-------

In [ ]:
# project_id = "YOUR_PROJECT_ID"
# model_id = "YOUR_MODEL_ID"
# file_path = "path_to_local_file.jpg"

file_path = "img_35.jpg"

prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = automl.AutoMlClient.model_path(project_id, "us-central1", model_id)

# Read the file.
with open(file_path, "rb") as content_file:
    content = content_file.read()

image = automl.Image(image_bytes=content)
payload = automl.ExamplePayload(image=image)

# params is additional domain-specific parameters.
# score_threshold is used to filter the result
params = {"score_threshold": "0.8"}

request = automl.PredictRequest(name=model_full_id, payload=payload, params=params)
response = prediction_client.predict(request=request)

print("Prediction results:")
for result in response.payload:
    print("Predicted class name: {}".format(result.display_name))
    print("Predicted class score: {}".format(result.classification.score))


In [13]:
#import joblib

In [18]:
#joblib.dump(response1.model[0].name.split("/")[-1] , 'usecase3model.pkl')

['usecase3model.pkl']

In [23]:
#%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [24]:
#%reload_ext google.cloud.bigquery

In [25]:
#from google.cloud import bigquery

#client = bigquery.Client()

In [5]:
from google.cloud import storage

client = storage.Client("gebu-data-ml-day0-01-333910")
uri_data = "https://storage.cloud.google.com/ml_usecase_3/{}?authuser=7"
for blob in client.list_blobs('ml_usecase_3', prefix='validation'):
    if str(blob.name) != "validation/":
        file_name = "gs://" + str(blob.name)
        url_data = uri_data.format(str(blob.name))
        print(file_name,url_data)
   

gs://validation/cat.19.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.19.jpg?authuser=7
gs://validation/cat.20.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.20.jpg?authuser=7
gs://validation/cat.21.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.21.jpg?authuser=7
gs://validation/cat.22.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.22.jpg?authuser=7
gs://validation/cat.23.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.23.jpg?authuser=7
gs://validation/cat.24.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.24.jpg?authuser=7
gs://validation/cat.25.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.25.jpg?authuser=7
gs://validation/cat.26.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.26.jpg?authuser=7
gs://validation/cat.27.jpg https://storage.cloud.google.com/ml_usecase_3/validation/cat.27.jpg?authuser=7
gs://validation/cat.28.jpg https://storage.clo